In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel


X_train = np.load('initial_data/function_7/initial_inputs.npy')
Y_train = np.load('initial_data/function_7/initial_outputs.npy')

#[0.020402 0.434337 0.300466 0.170135 0.349566 0.743187]
X_train = np.append(X_train, [[0.020402,0.434337,0.300466,0.170135,0.349566,0.743187],
                              [0,0.340848,0.345509,0.051618,0.312636,0.85057 ],
                              [0,0.429559,0.340164,0.16567,0.313112,0.624914],
                              [0,0.337135,0.24888,0.217358,0.266991,0.636595],
                              [0,0.36127,0.121505,0.142677,0.200176,0.600516],
                              [0,0.245747,0.336856,0.316917,0.317564,0.651608]], axis=0)
Y_train = np.append(Y_train, [1.6762583840644714,1.10533948770441,1.80623751635318,
                              2.100936293593643,1.296158521067342,2.587893599419789])

print(X_train)
print(Y_train)

[[0.27262382 0.32449536 0.89710881 0.83295115 0.15406269 0.79586362]
 [0.54300258 0.9246939  0.34156746 0.64648585 0.71844033 0.34313266]
 [0.09083225 0.66152938 0.06593091 0.25857701 0.96345285 0.6402654 ]
 [0.11886697 0.61505494 0.90581639 0.8553003  0.41363143 0.58523563]
 [0.63021764 0.8380969  0.68001305 0.73189509 0.52673671 0.34842921]
 [0.76491917 0.25588292 0.60908422 0.21807904 0.32294277 0.09579366]
 [0.05789554 0.49167222 0.24742222 0.21811844 0.42042833 0.73096984]
 [0.19525188 0.07922665 0.55458046 0.17056682 0.01494418 0.10703171]
 [0.64230298 0.83687455 0.02179269 0.10148801 0.68307083 0.6924164 ]
 [0.78994255 0.19554501 0.57562333 0.07365919 0.25904917 0.05109986]
 [0.52849733 0.45742436 0.36009569 0.36204551 0.81689098 0.63747637]
 [0.72261522 0.01181284 0.06364591 0.16517311 0.07924415 0.35995166]
 [0.07566492 0.33450212 0.13273274 0.60831236 0.91838592 0.82233079]
 [0.94245084 0.37743962 0.48612233 0.22879108 0.08263175 0.71195755]
 [0.14864702 0.03394336 0.72880565

In [2]:
# Define Gaussian Process
kernel = Matern(nu=2.5) + WhiteKernel(noise_level=1)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-5, normalize_y=True)
gp.fit(X_train, Y_train.ravel())

C:\Code\RoughWork\Jupyter\venv\lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(alpha=1e-05,
                         kernel=Matern(length_scale=1, nu=2.5) + WhiteKernel(noise_level=1),
                         normalize_y=True)

In [3]:
def ucb(X, gp, kappa=1.96):  # kappa can be adjusted based on your confidence level needs
    mean, std = gp.predict(X, return_std=True)
    return mean + kappa * std


In [4]:
from scipy.optimize import minimize

def propose_point(gp, bounds=np.array([[0, 1]]*6), n_restarts=10):
    best_x = None
    best_ucb = float('-inf')

    for starting_point in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, 6)):
        res = minimize(lambda x: -ucb(x.reshape(1, -1), gp), x0=starting_point, bounds=bounds, method='L-BFGS-B')
        if -res.fun > best_ucb:
            best_ucb = -res.fun
            best_x = res.x
            
    return best_x

new_point = propose_point(gp)
print("Proposed new hyperparameters:", new_point)

print(np.array2string(new_point,precision=6,separator='-',floatmode='fixed',formatter={'float':'{:0.6f}'.format}))

Proposed new hyperparameters: [0.         0.13568925 0.29997878 0.40769508 0.27667462 0.7210259 ]
[0.000000-0.135689-0.299979-0.407695-0.276675-0.721026]
